In [1]:
from dotenv import load_dotenv
import os

# Load from .env
load_dotenv()

#os.environ.get("OPENAI_API_KEY")

True

### Connect to OpenAI

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
output = llm.invoke('Explain quantum mechancis in one sentence')
print(output.content)

Quantum mechanics is the branch of physics that describes the behavior of matter and energy on the smallest scales, using principles such as superposition, entanglement, and quantized energy levels.


In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()  

# invoking the llm (running the prompt)
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo', temperature=0.1)
print(output.content)

Quantum mechanics is the branch of physics that describes the behavior of particles at the smallest scales, where traditional laws of physics no longer apply and phenomena such as superposition and entanglement occur.


In [5]:
# using Chat Completions API Messages: System, Assistant and Human
from langchain.schema import(
    SystemMessage, 
    AIMessage,
    HumanMessage
)
messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf atomarer und subatomarer Ebene durch probabilistische Wellenfunktionen.


## Caching LLM Responses
Storing frequently access data in a temporary, faster storage layer

### 1. In-Memory Cache

In [9]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [11]:
prompt = 'Tell a me a joke that a toddler can understand.'
print(llm.invoke(prompt))



Why couldn't the bicycle stand up by itself? Because it was two-tired!


In [12]:
prompt = 'Tell a me a joke that a toddler can understand.'
print(llm.invoke(prompt))



Why did the cookie go to the doctor?

Because it was feeling crumbly!


In [13]:
prompt = 'Tell a me a joke that a toddler can understand.'
print(llm.invoke(prompt))



Why was the little strawberry crying?

Because it was in a jam!


In [14]:
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

In [15]:
%%time
prompt = 'Tell a me a joke that a toddler can understand.'
llm.invoke(prompt)

CPU times: total: 31.2 ms
Wall time: 1.18 s


' \n\nWhy was the math book sad?\nBecause it had too many problems.'

In [16]:
%%time
prompt = 'Tell a me a joke that a toddler can understand.'
llm.invoke(prompt)

CPU times: total: 0 ns
Wall time: 0 ns


' \n\nWhy was the math book sad?\nBecause it had too many problems.'

In [17]:
%%time
prompt = 'Tell a me a joke that a toddler can understand.'
llm.invoke(prompt)

CPU times: total: 0 ns
Wall time: 0 ns


' \n\nWhy was the math book sad?\nBecause it had too many problems.'

### 2. SQLite Caching

In [18]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [19]:
%%time
# First request (not in cache, takes longer)
llm.invoke("Tell me a joke")

CPU times: total: 31.2 ms
Wall time: 967 ms


" Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!"

In [20]:
%%time
# Second request (cached, faster)
llm.invoke("Tell me a joke")

CPU times: total: 0 ns
Wall time: 5.04 ms


" Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!"

## LLM Streaming

In [21]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

Verse 1:
In the dead of night, under the pale moonlight
A raven soars, through the starry sky
Its feathers glisten, as it calls out to the night
A creature of mystery, with a piercing eye

Chorus:
Moon and Raven, dancing in the dark
Silent whispers, echoing in the park
Their spirits intertwined, forever bound
In the shadows, where they can be found

Verse 2:
The moon shines bright, as the raven takes flight
A watchful guardian, of the night
The darkness beckons, with a haunting song
A symphony of secrets, that doesn't belong

Chorus:
Moon and Raven, dancing in the dark
Silent whispers, echoing in the park
Their spirits intertwined, forever bound
In the shadows, where they can be found

Bridge:
The moon and the raven, a timeless pair
In the stillness of the night, they share a secret lair
With a knowing gaze, they understand
The beauty of the darkness, hand in hand

Chorus:
Moon and Raven, dancing in the dark
Silent whispers, echoing in the park
Their spirits intertwined, forever bound


In [22]:
llm.stream(prompt)

<generator object BaseChatModel.stream at 0x000002A071A19C40>

In [23]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

Verse 1:
Beneath the silver light of the moon
A raven flies, a creature of the night
Its wings glide through the darkness
As it calls out to the shadows

Chorus:
Oh moon, guide me through the night
With your gentle, haunting light
And raven, sing your eerie song
As we wander through the darkness long

Verse 2:
The moon whispers secrets to the stars
As the raven weaves a tale in the sky
Together they dance in the velvet night
A duet of darkness, a symphony of sight

Chorus:
Oh moon, guide me through the night
With your gentle, haunting light
And raven, sing your eerie song
As we wander through the darkness long

Bridge:
In the night, they are wild and free
A cosmic dance of destiny
The moon and raven, a mysterious pair
Their bond unbreakable, beyond compare

Chorus:
Oh moon, guide me through the night
With your gentle, haunting light
And raven, sing your eerie song
As we wander through the darkness long

Outro:
As the night fades into the dawn
The moon and raven bid farewell
But their s

## PromptTemplates

In [24]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define a template for the prompt
template = '''You are an experienced virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template=template)

# Fill in the variable: virus and language
prompt = prompt_template.format(virus='hiv', language='german')
prompt  # Returns the generated prompt


'You are an experienced virologist.\nWrite a few sentences about the following virus "hiv" in german.'

In [25]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

HIV, das humane Immundefizienzvirus, ist ein Retrovirus, das das menschliche Immunsystem schwächt und zu AIDS führen kann. Es wird hauptsächlich durch den Kontakt mit infiziertem Blut, sexuellen Kontakt oder von der Mutter auf das Kind während der Schwangerschaft übertragen. Es ist wichtig, sich über die Übertragungswege und Präventionsmaßnahmen von HIV zu informieren, um die Verbreitung der Krankheit einzudämmen. Die Forschung und Entwicklung von antiretroviralen Medikamenten haben dazu beigetragen, die Lebenserwartung von HIV-positiven Menschen zu verbessern.


## ChatPromptTemplates

In [26]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

# Fill in the specific values for n and area
messages = chat_template.format_messages(n='5', area='World')
print(messages)  # Outputs the formatted chat messages

[SystemMessage(content='You respond only in the JSON format.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Top 5 countries in World by population.', additional_kwargs={}, response_metadata={})]


In [27]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

{
    "countries": [
        {
            "rank": 1,
            "country": "China",
            "population": 1439323776
        },
        {
            "rank": 2,
            "country": "India",
            "population": 1380004385
        },
        {
            "rank": 3,
            "country": "United States",
            "population": 331002651
        },
        {
            "rank": 4,
            "country": "Indonesia",
            "population": 273523615
        },
        {
            "rank": 5,
            "country": "Pakistan",
            "population": 220892331
        }
    ]
}


## Simple Chains
1. Receives input (e.g., a question or keyword)
2. Fills in a prompt using that input
3. Sends the prompt to an LLM
4. Returns the LLM’s output

In [28]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experience virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)


In [29]:
chain = LLMChain(
    llm = llm,
    prompt=prompt_template,
    verbose=True
)
output = chain.invoke({'virus':'HSV','language':'German'})
print(output)

C:\Users\andi\AppData\Local\Temp\ipykernel_30136\4165081667.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(




> Entering new LLMChain chain...
Prompt after formatting:
You are an experience virologist.
Write a few sentences about the following virus "HSV" in German.

> Finished chain.
{'virus': 'HSV', 'language': 'German', 'text': 'HSV steht für das Herpes-simplex-Virus, das in zwei Typen unterteilt ist: HSV-1 und HSV-2. HSV-1 verursacht in der Regel Lippenherpes, während HSV-2 hauptsächlich für Genitalherpes verantwortlich ist. Herpesviren bleiben lebenslang im Körper und können in stressigen Situationen oder bei einem geschwächten Immunsystem reaktiviert werden. Es gibt antivirale Medikamente, die dabei helfen können, den Verlauf und die Symptome von HSV-Infektionen zu lindern.'}


In [30]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points'
prompt_template = PromptTemplate.from_template(template=template)
# Initialize an LLMChain with the ChatOpenAI model and the prompt template
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

In [31]:
country = input('Enter Country: ')

# Invoke the chain with specific virus and language values
output = chain.invoke(country)
print(output['text'])



> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of italy?. List the top 3 places to visit in that city. Use bullet points

> Finished chain.
The capital of Italy is Rome.

Top 3 places to visit in Rome:
- Colosseum
- Vatican City (including St. Peter's Basilica and the Sistine Chapel)
- Trevi Fountain


### Sequential Chains
- make a series of call to one or more LLMs.
- the output fro one chain will be the input to another chain

In [32]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Initialize the first ChatOpenAI model (gpt-3.5-turbo) with specific temperature
llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

# Define the first prompt template
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)
# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

# Initialize the second ChatOpenAI model (gpt-4-turbo) with specific temperature
llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)

# Define the second prompt template
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
# Create another LLMChain using the second model and the prompt template
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

In [33]:
# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('linear regression')



> Entering new SimpleSequentialChain chain...
Sure, here is a simple implementation of linear regression in Python:

```python
import numpy as np

def linear_regression(X, y):
    # Add a column of ones to X for the intercept term
    X = np.c_[np.ones(X.shape[0]), X]
    
    # Calculate the coefficients using the normal equation method
    theta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
    
    return theta

# Example usage
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 4, 6, 8, 10])

theta = linear_regression(X, y)
print(theta)
```

This function takes two numpy arrays `X` and `y` as input, where `X` contains the input features and `y` contains the target values. It adds a column of ones to `X` to account for the intercept term, then calculates the coefficients `theta` using the normal equation method. Finally, it returns the coefficients `theta` which represent the slope and intercept of the linear regression model.

You can use this function to fit a linear regressi

In [34]:
# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('deep learning')



> Entering new SimpleSequentialChain chain...
Sure, here is an example of a simple deep learning function implemented in Python using the TensorFlow library:

```python
import tensorflow as tf

def deep_learning_model(input_shape, num_classes):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model
```

This function creates a simple deep learning model with three fully connected layers. The first layer flattens the input data, the second and third layers are dense layers with ReLU activation functions, and the final layer is a dense layer with a softmax activation function for multi-class classification.

In [35]:
print(output['output'])

The Python function `deep_learning_model`, implemented using TensorFlow, particularly with its high-level API Keras, is designed to ease the creation of deep neural network models geared towards classification tasks. Here’s a step-by-section breakdown and detailed analysis of what the function does, the deep learning concepts involved, and how it can be utilized:

### Function Signature
```python
def deep_learning_model(input_shape, num_classes):
```
- **Purpose**: It generates a sequential deep learning model based on the specified `input_shape` of the data and the `num_classes` for classification.
- **Parameters**:
  - `input_shape`: A tuple that indicates the shape of the input data. This should exclude the batch size dimension and only include the dimensions of the data representation. For example, in image recognition tasks, this could be the height, width, and depth (channels) of the images.
  - `num_classes`: An integer denoting the number of unique classes for the output predic